In [1]:
cd("/home/gridsan/aramadhan/Oceananigans.jl/")
pwd()

"/home/gridsan/aramadhan/Oceananigans.jl"

In [2]:
using Pkg
Pkg.activate(".")

"/home/gridsan/aramadhan/Oceananigans.jl/Project.toml"

In [1]:
using Oceananigans, Oceananigans.Operators

┌ Info: Recompiling stale cache file /home/gridsan/aramadhan/.julia/compiled/v1.1/Oceananigans/hU93i.ji for Oceananigans [9e8cae18-63c1-5223-a75c-80ca9d6e9a09]
└ @ Base loading.jl:1184


CUDA-enabled GPU(s) detected:
CuDevice(0): Tesla V100-SXM2-16GB


In [2]:
using Random, Statistics
using FFTW

In [5]:
Nx, Ny, Nz = 8, 8, 8
Lx, Ly, Lz = 1, 1, 1
R = rand(MersenneTwister(0), Nx, Ny, Nz)
R = R .- mean(R)

8×8×8 Array{Float64,3}:
[:, :, 1] =
  0.319189  -0.14263     0.0714293  …   0.447233  -0.320343    0.043884 
  0.405898   0.468758    0.363821       0.29666   -0.192313   -0.0275754
 -0.339892   0.0813535   0.463341      -0.380135  -0.308051    0.0979165
 -0.327129   0.0348312   0.263232      -0.390189   0.369123    0.286888 
 -0.225578  -0.244422   -0.0351544     -0.424904   0.150463   -0.455336 
 -0.300982   0.405588   -0.442091   …   0.272216   0.0822544  -0.0216966
 -0.462156  -0.337422   -0.151329      -0.399635   0.128237    0.160793 
 -0.436189   0.15099     0.263144       0.333617   0.230546    0.337798 

[:, :, 2] =
 -0.335351    -0.112398   -0.409666  …  -0.267658   -0.276227   -0.0693668 
  0.434399    -0.0251016   0.444426     -0.430831   -0.246833    0.482016  
  0.279268    -0.436235    0.27073      -0.39124     0.295758    0.492683  
  0.00134654  -0.02451    -0.327049      0.380707    0.211811    0.00313643
  0.117698    -0.0360611  -0.424435      0.0102109   0.0241184 

In [6]:
R2 = copy(R)
R2 .= cat(R2[:, 1:2:Ny, :], R2[:, Ny:-2:2, :]; dims=2)
R2 .= cat(R2[:, :, 1:2:Nz], R2[:, :, Nz:-2:2]; dims=3);

In [7]:
@inline W(M, k) = exp(-2im*π*k/M)

ω_4Ny_ky⁺ = reshape(W.(4Ny, 0:Ny-1), 1, Ny, 1)
ω_4Ny_ky⁻ = reshape(W.(4Ny, 0:-1:-(Ny-1)), 1, Ny, 1)
ω_4Nz_kz⁺ = reshape(W.(4Nz, 0:Nz-1), 1, 1, Nz)
ω_4Nz_kz⁻ = reshape(W.(4Nz, 0:-1:-(Nz-1)), 1, 1, Nz)

R3 = fft(R2, [2, 3])

r_inds = [1, collect(Ny:-1:2)...]
R3⁻ = view(R3, 1:Nx, r_inds, 1:Nz)

R4 = zeros(Nx, Ny, Nz)
@. R4 = 2 * real(ω_4Nz_kz⁺ * (ω_4Ny_ky⁺ * R3 + ω_4Ny_ky⁻ * R3⁻));

In [8]:
R4 ≈ FFTW.r2r(R, FFTW.REDFT10, [2, 3])

true

In [9]:
R4 = fft(R4, 1)

8×8×8 Array{Complex{Float64},3}:
[:, :, 1] =
 -2.66454e-14+0.0im        16.3717+0.0im      …   8.12933+0.0im    
      19.5186+4.81314im    12.2361-13.9949im     0.595239-22.4995im
       3.1991+13.9879im  -0.498285-11.4825im     -13.0339+14.8646im
      -14.617-15.9059im   -5.18881-12.2657im     -3.82183-6.10438im
      -7.1959+0.0im       -16.3921+0.0im           17.487+0.0im    
      -14.617+15.9059im   -5.18881+12.2657im  …  -3.82183+6.10438im
       3.1991-13.9879im  -0.498285+11.4825im     -13.0339-14.8646im
      19.5186-4.81314im    12.2361+13.9949im     0.595239+22.4995im

[:, :, 2] =
 -16.0768+0.0im      -4.38193+0.0im      …   1.95911+0.0im    
   6.2655-4.10268im   8.08108-8.56454im     -12.0896-16.5191im
 -22.4994+9.64744im   6.04621-8.56363im      6.15343+13.6236im
 -10.5211-2.95195im  -10.9848-2.51886im     -4.35911+8.02747im
 -24.9668+0.0im      -2.12844+0.0im         -5.84153+0.0im    
 -10.5211+2.95195im  -10.9848+2.51886im  …  -4.35911-8.02747im
 -22.4994-9.64744im 

In [10]:
is = reshape(1:Nx, Nx, 1, 1)
js = reshape(1:Ny, 1, Ny, 1)
ks = reshape(1:Nz, 1, 1, Nz)

kx² = @. (2sin((is-1)*π/Nx)    / (Lx/Nx))^2
ky² = @. (2sin((js-1)*π/(2Ny)) / (Ly/Ny))^2
kz² = @. (2sin((ks-1)*π/(2Nz)) / (Lz/Nz))^2

@. R4 = -R4 / (kx² + ky² + kz²)

R4[1, 1, 1] = 0

R4

8×8×8 Array{Complex{Float64},3}:
[:, :, 1] =
       0.0+0.0im          -1.68029-0.0im        …   -0.0330116-0.0im      
 -0.520631-0.128384im    -0.259054+0.296291im      -0.00209778+0.0792943im
 -0.024993-0.109281im   0.00361749+0.0833612im       0.0348262-0.0397176im
 0.0668941+0.0727925im   0.0227327+0.0537372im      0.00822313+0.0131343im
  0.028109-0.0im         0.0616841-0.0im            -0.0348168-0.0im      
 0.0668941-0.0727925im   0.0227327-0.0537372im  …   0.00822313-0.0131343im
 -0.024993+0.109281im   0.00361749-0.0833612im       0.0348262+0.0397176im
 -0.520631+0.128384im    -0.259054-0.296291im      -0.00209778-0.0792943im

[:, :, 2] =
   1.65001-0.0im          0.224866-0.0im        …  -0.00765279-0.0im      
 -0.132649+0.0868591im    -0.14183+0.150315im        0.0411924+0.0562849im
  0.163343-0.0700392im  -0.0409949+0.0580637im      -0.0160246-0.035478im 
 0.0460938+0.0129328im   0.0461551+0.0105836im      0.00918656-0.0169174im
 0.0939509-0.0im        0.00772609-0.0im  

In [11]:
R4 = ifft(R4, 1)

8×8×8 Array{Complex{Float64},3}:
[:, :, 1] =
  -0.116169-1.73472e-17im  …    0.00175933+2.60209e-18im
 -0.0702269-3.85338e-17im      -0.00800866+3.32063e-18im
  0.0600559-3.46945e-17im       -0.0337251+8.67362e-19im
  0.0828541-2.03447e-17im       -0.0242185-3.32063e-18im
     0.1107+3.46945e-18im      -0.00130334-4.33681e-18im
    0.11784+3.50643e-17im  …     0.0283188-1.58591e-18im
 -0.0405321+4.85723e-17im     -0.000645105+8.67362e-19im
  -0.144522+2.38141e-17im       0.00481096+1.58591e-18im

[:, :, 2] =
 0.237193+3.46945e-18im  …  0.00905816+1.73472e-18im
 0.162779+7.95507e-18im     0.00518537+1.40627e-18im
 0.158678-6.93889e-18im     -0.0138249+1.73472e-18im
 0.190955-2.26747e-17im     -0.0238694+1.047e-18im  
  0.28047-1.04083e-17im     -0.0161313-1.73472e-18im
 0.261256+1.28616e-17im  …  0.00778813-2.27363e-18im
 0.195641+1.38778e-17im      0.0227763-1.73472e-18im
 0.163041+1.85801e-18im     0.00136489-1.79636e-19im

[:, :, 3] =
 -0.0932523-8.67362e-19im  …  -0.00612046+8.80914

In [12]:
ω_4Ny_ky⁺ = reshape(W.(4Ny, 0:Ny-1), 1, Ny, 1)
ω_4Ny_ky⁻ = reshape(W.(4Ny, 0:-1:-(Ny-1)), 1, Ny, 1)
ω_4Nz_kz⁺ = reshape(W.(4Nz, 0:Nz-1), 1, 1, Nz)
ω_4Nz_kz⁻ = reshape(W.(4Nz, 0:-1:-(Nz-1)), 1, 1, Nz)

r_y_inds = [1, collect(Ny:-1:2)...]
r_z_inds = [1, collect(Nz:-1:2)...]

R4⁻⁺ = view(R4, 1:Nx, r_y_inds, 1:Nz)
R4⁺⁻ = view(R4, 1:Nx, 1:Ny, r_z_inds)
R4⁻⁻ = view(R4, 1:Nx, r_y_inds, r_z_inds)

mask_ky = ones(1, Ny, 1); mask_ky[1] = 0;
mask_kz = ones(1, 1, Nz); mask_kz[1] = 0;

R5 = zeros(Complex{Float64}, Nx, Ny, Nz)
@. R5 = 1/4 *  ω_4Ny_ky⁻ * ω_4Nz_kz⁻ * ((R4 - mask_ky * mask_kz * R4⁻⁻) - im*(mask_kz * R4⁺⁻ + mask_ky * R4⁻⁺))

8×8×8 Array{Complex{Float64},3}:
[:, :, 1] =
 -0.0290422-4.33681e-18im  …  -0.0637881+0.0131367im 
 -0.0175567-9.63344e-18im     -0.0862771+0.0151202im 
   0.015014-8.67362e-18im     -0.0663443+0.00460025im
  0.0207135-5.08618e-18im     -0.0524197+0.00425369im
  0.0276749+8.67362e-19im     -0.0349649+0.00662274im
    0.02946+8.76608e-18im  …  -0.0297334+0.0131327im 
  -0.010133+1.21431e-17im     -0.0349943+0.00679636im
 -0.0361304+5.95354e-18im      -0.045088+0.0101949im 

[:, :, 2] =
 0.0596959+0.00384075im   …  -0.00133789-0.00200612im
 0.0404635+0.00517111im       -0.0117597-2.05682e-5im
 0.0392038+0.00624844im      -0.00317411-0.00100566im
 0.0477609+0.00459025im       0.00743793-0.00389522im
 0.0686976+0.0140443im        0.00510589-0.00142718im
  0.064407+0.0109929im    …    0.0188606+0.00305531im
 0.0493198+0.00275867im        0.0199752+0.00395622im
 0.0414435+0.000580159im       0.0112155-0.00410216im

[:, :, 3] =
  -0.0231579-0.00501204im  …  0.0165894-0.00165381im 
  -0.023044

In [16]:
R6 = ifft(R5, [2, 3])

8×8×8 Array{Complex{Float64},3}:
[:, :, 1] =
 -0.00159679+6.59789e-19im  …  -0.000855049+3.40574e-19im
  -0.0033367-8.40861e-20im      -0.00331986+3.31067e-20im
 0.000688929-8.47117e-19im     -0.000804495-6.73274e-19im
  0.00297018-7.92802e-19im       0.00116887-8.47618e-19im
  0.00390202-4.53622e-19im       0.00290069-3.74726e-19im
  0.00429916-1.91481e-19im  …    0.00162123+5.80227e-20im
  0.00431429-1.01838e-19im       0.00244499+2.79947e-19im
  0.00250602+6.54848e-19im      -0.00016344+4.50729e-19im

[:, :, 2] =
 0.000485045+4.43181e-19im  …   -0.00201165+1.03537e-19im
 -0.00083052+7.18957e-20im      -0.00284027+1.59917e-19im
 -0.00185301-3.07173e-19im     -0.000305006-4.93372e-19im
 0.000675598-5.26609e-19im     -0.000800465-6.42926e-19im
  0.00213595-3.25448e-19im     -0.000185119-1.20705e-19im
  0.00155191+3.36472e-19im  …    0.00176866-1.44757e-20im
 0.000621566+4.05608e-19im      0.000912773+3.38419e-19im
 0.000729863+5.67769e-19im      -0.00174163+5.67802e-19im

[:, :, 3] =
 

In [20]:
R7 = zeros(Nx, Ny, Nz)

p_y_inds = [1:2:Ny..., Ny:-2:2...]
p_z_inds = [1:2:Nz..., Nz:-2:2...]
R7_p = view(R7, 1:Nx, p_y_inds, p_z_inds)

@. R7_p = real(R6)

8×8×8 view(::Array{Float64,3}, 1:8, [1, 3, 5, 7, 8, 6, 4, 2], [1, 3, 5, 7, 8, 6, 4, 2]) with eltype Float64:
[:, :, 1] =
 -0.00159679   -0.000847561   0.0010956    …   0.00150972   -0.000855049
 -0.0033367    -0.00244706    0.00145344       0.0015242    -0.00331986 
  0.000688929  -0.00166385    0.00356681       0.00288025   -0.000804495
  0.00297018    0.000428707   0.00151528       0.00293817    0.00116887 
  0.00390202    0.00229001    0.00121111       0.00193082    0.00290069 
  0.00429916    0.00315583    0.00308235   …   0.00297069    0.00162123 
  0.00431429    0.000937283   0.000915291     -0.000608081   0.00244499 
  0.00250602   -0.0015412    -0.00112641      -0.0016028    -0.00016344 

[:, :, 2] =
  0.000485045   0.000284204  0.000769018  …   0.00041781   -0.00201165 
 -0.00083052   -0.00184308   0.0022492        0.00107628   -0.00284027 
 -0.00185301   -0.00178016   0.00147633       0.000310334  -0.000305006
  0.000675598  -0.00114127   0.000394467      0.000132043  -0.0008

In [18]:
R7

8×8×8 Array{Float64,3}:
[:, :, 1] =
 -0.00159679   -0.000855049  -0.000847561  …   0.00228714    0.000870695
 -0.0033367    -0.00331986   -0.00244706       0.00293618    0.00138513 
  0.000688929  -0.000804495  -0.00166385       0.00304466    0.000834866
  0.00297018    0.00116887    0.000428707      0.000852907   0.000570942
  0.00390202    0.00290069    0.00229001       0.00166773    0.00332279 
  0.00429916    0.00162123    0.00315583   …   0.00118812    0.00109242 
  0.00431429    0.00244499    0.000937283      0.000205861  -0.00084154 
  0.00250602   -0.00016344   -0.0015412       -0.000486827  -0.00127385 

[:, :, 2] =
  0.000285914  -0.000576183   0.000211865  …  0.0031595    0.00177006 
 -0.00277691   -0.00183166   -0.00224354      0.00335707   0.000467899
 -0.00138409    0.000374582  -0.00113692      0.00152491  -0.000131318
  0.00100951    0.000893506   0.00152333      0.00128605   0.00175583 
  0.0019134     0.00170224    0.00248473      0.00208731   0.00284546 
  0.00265628

In [19]:
function ∇²!(grid::RegularCartesianGrid, f, ∇²f)
    for k in 1:grid.Nz, j in 1:grid.Ny, i in 1:grid.Nx
        @inbounds ∇²f[i, j, k] = ∇²(grid, f, i, j, k)
    end
end

grid = RegularCartesianGrid((Nx, Ny, Nz), (Lx, Ly, Lz))
fbcs = ChannelBCs()

ϕ   = CellField(Float64, CPU(), grid)
∇²ϕ = CellField(Float64, CPU(), grid)

data(ϕ) .= real.(R7)

fill_halo_regions!(grid, (:T, fbcs, ϕ.data))
∇²!(grid, ϕ, ∇²ϕ)

fill_halo_regions!(grid, (:T, fbcs, ∇²ϕ.data))

data(∇²ϕ) ≈ R

true